In [1]:
! pip install langchain langchain-community openai tiktoken pinecone-client langchain_pinecone unstructured pdfminer==20191125 pdfminer.six==20221105 pillow_heif unstructured_inference youtube-transcript-api pytube sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pdfplumber to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━

In [18]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, WebBaseLoader, YoutubeLoader, DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import numpy as np
import tiktoken
import os

In [5]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

In [8]:
openrouter_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=userdata.get("OPENROUTER_API")
)

In [9]:
tokenizer = tiktoken.get_encoding('p50k_base')

def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
)

In [10]:
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def get_embedding(text):
    return hf_embeddings.embed_query(text)

def cosine_similarity_between_sentences(sentence1, sentence2):
    embedding1 = np.array(get_embedding(sentence1))
    embedding2 = np.array(get_embedding(sentence2))

    if embedding1 is None or embedding2 is None:
        return None

    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    print("Sentence 1 Embedding: ", embedding1)
    print("Sentence 2 Embedding: ", embedding2)

    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]

sentence1 = "Great to finally meet!"
sentence2 = "Nice to meet you"

similarity = cosine_similarity_between_sentences(sentence1, sentence2)
if similarity is not None:
    print(f"Cosine similarity between sentences:\n'{sentence1}' and '{sentence2}': {similarity:.4f}")
else:
    print("Could not compute similarity due to an error.")

<ipython-input-10-8fa2e95f6ca8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Sentence 1 Embedding:  [[-6.42226040e-02 -2.30498277e-02  4.12189811e-02  5.67926466e-02
  -1.63038597e-02  4.15017158e-02  8.88479128e-03 -4.60831709e-02
  -1.57466414e-03 -2.54955254e-02 -3.06746550e-03  9.07608494e-03
   1.05423806e-02  3.58030386e-02  3.46321873e-02  9.65092238e-03
  -2.56988853e-02 -3.51948030e-02  3.15498598e-02  3.47745456e-02
  -5.27179986e-02 -2.66816318e-02  2.66268365e-02 -1.34273320e-02
  -5.38467579e-02  3.75799456e-04 -2.99408399e-02  4.46607508e-02
   4.49724793e-02  1.95614807e-02 -1.07248917e-01  1.15735307e-01
  -4.73707877e-02  5.04956283e-02  4.24734168e-02  3.78863029e-02
   3.02728396e-02 -3.62443663e-02  7.66253024e-02  1.54855186e-02
  -3.44800763e-02  1.85779128e-02  5.72161041e-02  5.62272444e-02
  -3.45480926e-02  4.13312130e-02  5.84770320e-03  4.52666581e-02
   1.93616562e-02  2.63053197e-02  2.68412270e-02  1.28470203e-02
   3.65549289e-02 -4.26418483e-02  2.21714508e-02  1.24915272e-01
  -1.69741251e-02 -1.28599599e-01 -3.78995319e-03 -1.

In [11]:
loader = YoutubeLoader.from_youtube_url("https://youtu.be/Br7AcznvzSA?feature=shared", add_video_info=True)
data = loader.load()
print(data)

[Document(metadata={'source': 'Br7AcznvzSA', 'title': 'How to create Hugging Face Access Token Key | Generative AI | Data Magic AI #HuggignFaceKey', 'description': 'Unknown', 'view_count': 5464, 'thumbnail_url': 'https://i.ytimg.com/vi/Br7AcznvzSA/hq720.jpg', 'publish_date': '2024-03-21 00:00:00', 'length': 238, 'author': 'Data Magic (by Sunny Kusawa)'}, page_content="Hello friends welcome back I'm sikman in this session we are going to talk about how you can create a hugging face token Keys the similar way you have a token keys for accessing a different apis like a platforms like open AI or you access like a Google API key similar way when you want to access a free open source models from the hugging phase so for few models you might need a hugging phase key so you can access those models so I'm going to show you how to create a hugging face key on the hugging face platform so first step you have to visit like huggingface doco this is the official website once you hit on this website 

In [12]:
texts = text_splitter.split_documents(data)

In [13]:
texts

[Document(metadata={'source': 'Br7AcznvzSA', 'title': 'How to create Hugging Face Access Token Key | Generative AI | Data Magic AI #HuggignFaceKey', 'description': 'Unknown', 'view_count': 5464, 'thumbnail_url': 'https://i.ytimg.com/vi/Br7AcznvzSA/hq720.jpg', 'publish_date': '2024-03-21 00:00:00', 'length': 238, 'author': 'Data Magic (by Sunny Kusawa)'}, page_content="Hello friends welcome back I'm sikman in this session we are going to talk about how you can create a hugging face token Keys the similar way you have a token keys for accessing a different apis like a platforms like open AI or you access like a Google API key similar way when you want to access a free open source models from the hugging phase so for few models you might need a hugging phase key so you can access those models so I'm going to show you how to create a hugging face key on the hugging face platform so first step you have to visit like huggingface doco this is the official website once you hit on this website 

In [36]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key
pc = Pinecone()
pc.create_index(
    name="workshop", dimension=384, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [37]:
vectorstore = PineconeVectorStore(index_name="workshop", embedding=hf_embeddings)

index_name = "workshop"

namespace = "youtube-video"

In [38]:
for document in texts:
  print("-----------")
  print(document.metadata, document.page_content)

-----------
{'source': 'Br7AcznvzSA', 'title': 'How to create Hugging Face Access Token Key | Generative AI | Data Magic AI #HuggignFaceKey', 'description': 'Unknown', 'view_count': 5464, 'thumbnail_url': 'https://i.ytimg.com/vi/Br7AcznvzSA/hq720.jpg', 'publish_date': '2024-03-21 00:00:00', 'length': 238, 'author': 'Data Magic (by Sunny Kusawa)', 'text': "Hello friends welcome back I'm sikman in this session we are going to talk about how you can create a hugging face token Keys the similar way you have a token keys for accessing a different apis like a platforms like open AI or you access like a Google API key similar way when you want to access a free open source models from the hugging phase so for few models you might need a hugging phase key so you can access those models so I'm going to show you how to create a hugging face key on the hugging face platform so first step you have to visit like huggingface doco this is the official website once you hit on this website login with yo

In [39]:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)

In [40]:
precomputed_embeddings = [hf_embeddings.embed_query(f"Source: {t.metadata['source']}, Title: {t.metadata['title']} \n\nContent: {t.page_content}") for t in texts]


In [41]:
for i, document in enumerate(texts):
    embedding = precomputed_embeddings[i]
    vectorstore.add_texts(
        texts=[document.page_content],
        metadatas=[document.metadata],
        embeddings=[embedding],
        namespace="youtube-video"
    )

In [42]:
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"))
pinecone_index = pc.Index("workshop")

In [75]:
def perform_youtube_rag(query):
    query_embedding = hf_embeddings.embed_query(query)

    top_matches = pinecone_index.query(vector=query_embedding, top_k=10, include_metadata=True, namespace=namespace)

    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n\n\n\n\nMY QUESTION:\n" + query

    system_prompt = f"""You are an expert personal assistant. Answer any questions I have about the Youtube Video provided. You always answer questions based only on the context that you have been provided.
    """

    res = openrouter_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
    )

    return res.choices[0].message.content

In [76]:
perform_youtube_rag("How to create Hugging Face Access Token Key?")

'To create a Hugging Face Access Token Key, you can follow these steps as outlined in the provided YouTube video:\n\n1. Visit the Hugging Face website and log in to your account.\n2. Go to your profile settings. You can usually find this by clicking on your profile picture.\n3. In the settings menu, look for the "Access Token" option.\n4. Click on "Access Token" to create a new token key.\n5. Provide a name for your key (e.g., for what project you need it) and select the access level (read, write, etc.).\n6. Click on the "Generate Token" button to create a new token key.\n7. Once the token key is generated, you can copy it to your clipboard.\n8. You can then paste this key into your code or store it in an environment variable for future use to access Hugging Face models or datasets.\n\nRemember, always keep your token key secure and do not share it with unauthorized users.'

In [72]:
loader = PyPDFLoader("file.pdf")
data = loader.load()

print(data)

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
    )

texts = text_splitter.split_documents(data)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

vectorstore_from_texts = PineconeVectorStore.from_texts(
    [f"Source: {t.metadata.get('source', 'Unknown')}, Title: {t.metadata.get('title', 'Unknown')} \n\nContent: {t.page_content}" for t in texts],
    embeddings,
    index_name=index_name,
    namespace=namespace
)

[Document(metadata={'source': 'file.pdf', 'page': 0}, page_content='CS390 – Information Security\nAssignment No. 04\nTitle: "Balancing Security and Privacy: Ethical Implications"\nNote: All submissions must be made on time through the Learning Management \nSystem (LMS). Additionally, while you are encouraged to utilize resources \nsuch as the internet and AI for research purposes, your work should \ndemonstrate authenticity and originality. It must be in your own words.\nDescription:\nThis assignment focuses on the delicate balance between ensuring \ninformation security and respecting individuals\' privacy rights. As \ntechnology continues to collect vast amounts of personal data, it\'s \nessential for information security professionals to navigate this \nlandscape ethically. You will examine the ethical implications of \nsurveillance, data collection, and privacy breaches in the context of \ninformation security practices. Document your findings in a report.\nTasks:\n- Investigate th

In [73]:
def perform_pdf_rag(query):
    query_embedding = hf_embeddings.embed_query(query)

    top_matches = pinecone_index.query(vector=query_embedding, top_k=10, include_metadata=True, namespace=namespace)

    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n\n\n\n\nMY QUESTION:\n" + query

    primer = "You are a personal assistant. Answer any questions I have about the PDF file provided."

    system_prompt = f"""You are an expert personal assistant. Answer any questions I have about the PDF file provided. You always answer questions based only on the context that you have been provided.
    """

    res = openrouter_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
    )

    return res.choices[0].message.content

In [74]:
perform_pdf_rag("What tasks we have to do?")

'The tasks outlined in the provided document for Assignment No. 04 titled "Balancing Security and Privacy: Ethical Implications" in CS390 – Information Security are as follows:\n\n1. Investigate the ethical considerations surrounding data collection and surveillance technologies.\n2. Analyze the trade-offs between security measures and privacy rights in different scenarios.\n3. Discuss the impact of data breaches on individuals\' privacy and trust.\n4. Examine regulatory frameworks and ethical guidelines relevant to protecting privacy in information security practices.\n5. Develop recommendations for organizations to uphold both security and privacy principles ethically.\n\nThese tasks are part of the assignment focusing on the delicate balance between information security and individuals\' privacy rights, emphasizing the ethical implications in the context of information security practices.'